In [23]:
import os, json
from collections import defaultdict
import customutils

alphapose_labels = customutils.get_classes('configs/halpe26_classes.txt')
openpose_labels = customutils.get_classes('configs/openpose_classes.txt')
vitpose_labels = customutils.get_classes('configs/cocowb_classes.txt')
cocoLabels = customutils.get_classes('configs/coco_classes.txt')
mpiiLabels = customutils.get_classes('configs/mpii_classes.txt')



In [24]:
def get_coco_pose(pose, labels, videoId):
    joints = defaultdict(list)
    imgId = int(pose['image_id'].replace('COCO_test2015_','').replace('COCO_val2014_','').replace('.jpg',''))
    if videoId > 0:
        imgId += videoId
    pose['image_id'] = imgId

    keypoints_index = 0
    keypoints = pose['keypoints']
    for label in labels:
        joints[label] = {'x': keypoints[keypoints_index], 'y': keypoints[keypoints_index+1], 'confidence': keypoints[keypoints_index+2]}
        keypoints_index += 3

    coco_keypoints = []
    for label in cocoLabels:
        coco_keypoints.append(joints[label]['x'])
        coco_keypoints.append(joints[label]['y'])
        coco_keypoints.append(2)
    pose['keypoints'] = coco_keypoints
    return pose

def get_mpii_pose(pose, labels):
    joints = defaultdict(list)
    imgId = int(pose['image_id'].replace('COCO_test2015_','').replace('COCO_val2014_','').replace('.jpg',''))
    pose['image_id'] = imgId

    keypoints_index = 0
    keypoints = pose['keypoints']
    for label in labels:
        if label in mpiiLabels:
            joints[label] = {'id': customutils.get_id_by_name(mpiiLabels, label), 'x': keypoints[keypoints_index], 'y': keypoints[keypoints_index+1], 'score': keypoints[keypoints_index+2]}
        keypoints_index += 3

    for label in mpiiLabels:
        if len(joints[label]) == 0:
            joints[label] = {'id': customutils.get_id_by_name(mpiiLabels, label) ,'x': 0, 'y': 0, 'score': 0}
    
    annopoints = {'annopoints': list(joints.values())}
    return annopoints

In [4]:
# COCO: AlphaPose
file_path = '../data/output/alphapose/coco/val2014/'
res_path = '../data/coco/results/alphapose/person_keypoints_val2014_2.json'
all_Results = []
full_file = file_path + 'alphapose-results.json'
# for filename in os.listdir(file_path):
    # full_file = os.path.join(file_path, filename) + '/alphapose-results.json'
with open(full_file, 'r') as f:
    predictions_annotations = json.load(f)
    for pose in predictions_annotations:      
        alphapose_pose = get_coco_pose(pose, alphapose_labels, -1)  
        alphapose_pose['score'] = 1
        all_Results.append(alphapose_pose)
    
    
customutils.writeJson(all_Results,res_path)

In [16]:
# MPII: AlphaPose
file_path = '../data/output/alphapose/mpii'
res_path = '../data/mpii/results/alphapose/preds.json'
all_Results = []
for filename in os.listdir(file_path):
    full_file = os.path.join(file_path, filename) + '/alphapose-results.json'
    with open(full_file, 'r') as f:
        predictions_annotations = json.load(f)
        annorect = []
        for pose in predictions_annotations:      
            annopoints = get_mpii_pose(pose, alphapose_labels)  
            annorect.append(annopoints)
        annolist = {'image_name': filename + '.jpg', 'annorect': annorect}
        all_Results.append(annolist)
    
    
customutils.writeJson(all_Results,res_path)

In [29]:
# COCO: ViTPose
file_path = '../data/output/ViTPose/coco/val2017/'
res_path = '../data/coco/results/ViTPose/person_keypoints_val2017.json'
all_Results = []
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
        full_file = os.path.join(file_path, filename)
        with open(full_file, 'r') as f:
            predictions_annotations = json.load(f)
            for pose in predictions_annotations:  
                vitpose_pose = get_coco_pose(pose, vitpose_labels, -1)
                vitpose_pose['score'] = 1
                vitpose_pose['category_id'] = 1
                all_Results.append(vitpose_pose)
    
    
customutils.writeJson(all_Results,res_path)

In [18]:
# MPII: ViTPose
file_path = '../data/output/ViTPose/mpii/'
res_path = '../data/mpii/results/ViTPose/preds.json'
all_Results = []
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
        full_file = os.path.join(file_path, filename)
        with open(full_file, 'r') as f:
            predictions_annotations = json.load(f)
            for pose in predictions_annotations:  
                annopoints = get_mpii_pose(pose, vitpose_labels)
                annorect.append(annopoints)
        annolist = {'image_name': filename.replace('.json', '.jpg'), 'annorect': annorect}
        all_Results.append(annolist)
    
    
customutils.writeJson(all_Results,res_path)

In [22]:
# COCO: OpenPose
file_path = '../data/output/openpose/coco/val2014'
res_path = '../data/coco/results/openpose/person_keypoints_val2014.json'
all_Results = []
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
      full_file = os.path.join(file_path, filename)
      with open(full_file, 'r') as f:
         predictions_annotations = json.load(f)
         for pose in predictions_annotations['people']:
            joints = defaultdict(list)
            imgId = int(filename.replace('COCO_test2015_','').replace('COCO_val2014_','').replace('_keypoints.json',''))

            keypoints_index = 0
            openpose_keypoints = pose['pose_keypoints_2d']
            for label in openpose_labels:
               joints[label] = {'x': openpose_keypoints[keypoints_index], 'y': openpose_keypoints[keypoints_index+1], 'confidence': openpose_keypoints[keypoints_index+2]}
               keypoints_index += 3

            coco_keypoints = []
            for label in cocoLabels:
               coco_keypoints.append(joints[label]['x'])
               coco_keypoints.append(joints[label]['y'])
               coco_keypoints.append(2)

            coco_pose = {'image_id': imgId, 'keypoints': coco_keypoints, 'category_id': 1, 'score': 1}
            all_Results.append(coco_pose)
    #break

customutils.writeJson(all_Results,res_path)

In [8]:
# MPII: OpenPose
file_path = '../data/output/openpose/mpii/json'
res_path = '../data/mpii/results/openpose/preds.json'
all_Results = []
for filename in os.listdir(file_path):
    full_file = os.path.join(file_path, filename)
    with open(full_file, 'r') as f:
        predictions_annotations = json.load(f)
        annorect = []
        for pose in predictions_annotations['people']:
           joints = defaultdict(list)
           imgId = filename.replace('_keypoints.json','.jpg')

           keypoints_index = 0
           openpose_keypoints = pose['pose_keypoints_2d']
           for label in openpose_labels:
              if label in mpiiLabels:
                  joints[label] = {'id': customutils.get_id_by_name(mpiiLabels, label) ,'x': openpose_keypoints[keypoints_index], 'y': openpose_keypoints[keypoints_index+1], 'score': openpose_keypoints[keypoints_index+2]}
              keypoints_index += 3

           for label in mpiiLabels:
              if len(joints[label]) == 0:
                  joints[label] = {'id': customutils.get_id_by_name(mpiiLabels, label) ,'x': 0, 'y': 0, 'score': 0}

           annopoints = {'annopoints': list(joints.values())}
           annorect.append(annopoints)

        annolist = {'image_name': imgId, 'annorect': annorect}
        all_Results.append(annolist)

customutils.writeJson(all_Results,res_path)

In [4]:
# Videos annotations

file_path = '../videos/annotations/'
res_path = '../videos/running_keypoints.json'
all_Results = []
video_count = 1000
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
        full_file = os.path.join(file_path, filename)
        with open(full_file, 'r') as f:
            predictions_annotations = json.load(f)
            file_name = predictions_annotations['item']['name']            
            frame_count = predictions_annotations['item']['slots'][0]['frame_count']
            for frame_index in range(0, frame_count):
                image_id = video_count + frame_index
                all_Results.append({'image_id': image_id, 'file_name': file_name, 'frame': frame_index})
        video_count += 1000
    
    
customutils.writeJson(all_Results,res_path)

In [35]:
# Videos annotations


def get_bb_coco(points):
    x_values = [item["x"] for item in points]
    y_values = [item["y"] for item in points]

    # Get the minimum x and y values
    min_x = min(x_values)
    max_x = max(x_values)
    min_y = min(y_values)
    max_y = max(y_values)

    box_width = max_x - min_x
    box_height = max_y - min_y

    return [min_x, min_y, box_width, box_height]
    

def compute_area_coco(bounding_box):
    return bounding_box[2]*bounding_box[3]

file_path = '../videos/annotations/'
res_path = '../videos/lite_running_keypoints.json'
coco_res_path = '../videos/coco_running_keypoints.json'
all_Results = []
coco_Results = []
video_count = 1000
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
        full_file = os.path.join(file_path, filename)
        with open(full_file, 'r') as f:
            predictions_annotations = json.load(f)
            file_name = predictions_annotations['item']['name']
            frame_count = predictions_annotations['item']['slots'][0]['frame_count']
            first = True
            for annot in predictions_annotations['annotations']:
                
                for frame_index in range(0, frame_count):
                    if frame_index < len(annot['frames']):
                        points = annot['frames'][str(frame_index)]['skeleton']['nodes']
                        bb = get_bb_coco(points)
                        area = compute_area_coco(bb)

                        # Create a dictionary to map 'id' to its position in the order array
                        id_to_index = {id_value: index for index, id_value in enumerate(alphapose_labels)}

                        # Sort the data based on the custom sorting key
                        sorted_data = sorted(points, key=lambda x: id_to_index.get(x['name'], float('inf')))
                        kpts = []
                        for point in sorted_data:
                            if point['name'] in alphapose_labels: #check joint is available for this technique
                                kpts.append(point['x'])
                                kpts.append(point['y'])

                                if point["occluded"] == "false":
                                    kpts.append(2)
                                else:                       
                                    kpts.append(1)
                                    
                        image_id = video_count + frame_index
                        if first:
                            all_Results.append({'image_id': image_id, 'file_name': file_name, 'frame': frame_index})
                            
                        coco_Results.append({'id': image_id, 'image_id': image_id, "category_id": 1, "bbox": bb, "area": area, "iscrowd": 1, "num_keypoints": len(alphapose_labels), "keypoints": kpts})

                first = False
        video_count += 1000
    
    
customutils.writeJson(all_Results,res_path)
customutils.writeJson({"annotations":coco_Results},coco_res_path)

In [36]:
import pandas as pd
running_annotations = '../videos/lite_running_keypoints.json'
with open(running_annotations, 'r') as f:
    videos_ids = defaultdict(list)
    annots = json.load(f)
    df_running_annotations = pd.DataFrame(annots)

print(df_running_annotations)

    

      image_id                                         file_name  frame
0         1000               Athletics_Mixed_Tokyo_2020_20_1.mp4      0
1         1001               Athletics_Mixed_Tokyo_2020_20_1.mp4      1
2         1002               Athletics_Mixed_Tokyo_2020_20_1.mp4      2
3         1003               Athletics_Mixed_Tokyo_2020_20_1.mp4      3
4         1004               Athletics_Mixed_Tokyo_2020_20_1.mp4      4
...        ...                                               ...    ...
5586     46160  World_Athletics_Women_Marathon_Oregon_2022_8.mp4    160
5587     46161  World_Athletics_Women_Marathon_Oregon_2022_8.mp4    161
5588     46162  World_Athletics_Women_Marathon_Oregon_2022_8.mp4    162
5589     46163  World_Athletics_Women_Marathon_Oregon_2022_8.mp4    163
5590     46164  World_Athletics_Women_Marathon_Oregon_2022_8.mp4    164

[5591 rows x 3 columns]


In [46]:
mask_frame = df_running_annotations['frame'] == 5
df = df_running_annotations.loc[mask_frame]
df.head()

,image_id,file_name,frame
5,1005,Athletics_Mixed_Tokyo_2020_20_1.mp4,5
71,2005,Athletics_Mixed_Tokyo_2020_39.mp4,5
151,3005,Athletics_Mixed_Tokyo_2020_5.mp4,5
312,4005,Athletics_Mixed_Tokyo_2020_64.mp4,5
534,5005,Athletics_Mixed_Tokyo_2020_8.mp4,5


In [ ]:
# Videos as COCO: AlphaPose
file_path = '../data/videos/results/alphapose/'
res_path = '../data/videos/results/alphapose_running_keypoints.json'
all_Results = []
for filename in os.listdir(file_path):
    full_file = os.path.join(file_path, filename) + '/alphapose-results.json'
    with open(full_file, 'r') as f:
        predictions_annotations = json.load(f)
        mask_file = df_running_annotations['file_name'] == filename + '.mp4'
        for pose in predictions_annotations:     
            mask_frame = df_running_annotations['frame'] == int(pose['image_id'].replace('.jpg', ''))
            alphapose_pose = get_coco_pose(pose, alphapose_labels, )  
            alphapose_pose['score'] = 1
            all_Results.append(alphapose_pose)
    
    
customutils.writeJson(all_Results,res_path)

In [5]:
# Videos as COCO: ViTPose
file_path = '../data/videos/results/ViTPose/'
res_path = '../data/videos/results/ViTPose_running_keypoints.json'
all_Results = []
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
        full_file = os.path.join(file_path, filename)
        with open(full_file, 'r') as f:
            predictions_annotations = json.load(f)
            for pose in predictions_annotations:  
                vitpose_pose = get_coco_pose(pose, vitpose_labels, -1)
                vitpose_pose['score'] = 1
                vitpose_pose['category_id'] = 1
                all_Results.append(vitpose_pose)
    
    
customutils.writeJson(all_Results,res_path)

In [ ]:
# Videos COCO: OpenPose
file_path = '../data/videos/results/openpose'
res_path = '../data/videos/results/openpose_person_keypoints_val2014.json'
all_Results = []
for filename in os.listdir(file_path):
    if filename.endswith('.json'):
      full_file = os.path.join(file_path, filename)
      with open(full_file, 'r') as f:
         predictions_annotations = json.load(f)
         for pose in predictions_annotations['people']:
            joints = defaultdict(list)
            imgId = int(filename.replace('COCO_test2015_','').replace('COCO_val2014_','').replace('_keypoints.json',''))

            keypoints_index = 0
            openpose_keypoints = pose['pose_keypoints_2d']
            for label in openpose_labels:
               joints[label] = {'x': openpose_keypoints[keypoints_index], 'y': openpose_keypoints[keypoints_index+1], 'confidence': openpose_keypoints[keypoints_index+2]}
               keypoints_index += 3

            coco_keypoints = []
            for label in cocoLabels:
               coco_keypoints.append(joints[label]['x'])
               coco_keypoints.append(joints[label]['y'])
               coco_keypoints.append(2)

            coco_pose = {'image_id': imgId, 'keypoints': coco_keypoints, 'category_id': 1, 'score': 1}
            all_Results.append(coco_pose)
    #break

customutils.writeJson(all_Results,res_path)